### Search on Coveralls and Sonar Cloud

* Search on these different services and record on 

1- Search on Coveralls:
* GET: https://coveralls.io/{repository_name}/{project_name}.json
* Ex.: https://coveralls.io/github/SonarSource/sonarqube.json

2- Search on SonarCloud:
* GET: https://sonarcloud.io/api/measures/search_history?component={project_name}&metrics=coverage
* Ex.: https://sonarcloud.io/api/measures/search_history?component=simgrid_simgrid&metrics=coverage

In [59]:
repos = getProjects()

for repo in repos:    
    project = repo[0]
    
    coveralls = searchOnCoveralls(project)
    sonar = searchOnSonar(project)
    service = None
    
    
    if coveralls and sonar:
        service = 'BOTH'
        #updateProjectMetric(project,'BOTH')
    elif coveralls:
        service = 'COVERALLS'
        #updateProjectMetric(project,'COVERALLS')
    elif sonar:
        service = 'SONAR'

    print('Project {} - Service: {}'.format(project,service))
    updateProjectMetric(project,service)

### Mine Test Coverage info

* Get Projects with coverage not mined
* Verify whether it uses Coveralls or Sonar Cloud
* Mine test coverage info on APIs

In [169]:
repos = getProjectsToMineCoverage()
i=1
for repo in repos:    
    project = repo[0]
    service = repo[1]
    sonar_id = repo[2]
    
    print('===== Project {}\t{}/{}'.format(project,i,len(repos)))
    i+=1
    
    if service == 'COVERALLS' or service == 'BOTH':
        #mineOnCoverallsByBuild(project)
        mineOnCoveralls(project)
    
    if service == 'SONAR' or service == 'BOTH':
        mineOnSonarCloud(project,sonar_id)

===== Project alibaba/druid	1/9
page 1
***** Project com.alibaba:druid	 1 coverage found on SONAR CLOUD.
===== Project alibaba/fastjson	2/9
page 1
***** Project com.alibaba:fastjson	 1 coverage found on SONAR CLOUD.
===== Project antlr/antlr4	3/9
page 1
***** Project Lotes_antlr4	 1 coverage found on SONAR CLOUD.
===== Project AsyncHttpClient/async-http-client	4/9
page 1
***** Project vijayvani_async-http-client	 1 coverage found on SONAR CLOUD.
===== Project Atmosphere/atmosphere	5/9
page 1
***** Project maharsuresh_atmosphere	 1 coverage found on SONAR CLOUD.
===== Project brianfrankcooper/YCSB	6/9
page 1
***** Project VijayEluri_YCSB	 1 coverage found on SONAR CLOUD.
===== Project fantom-foundation/go-lachesis	7/9
page 1
***** Project Fantom-foundation_go-lachesis	 12 coverage found on SONAR CLOUD.
===== Project openhab/openhab1-addons	8/9
page 1
***** Project openhab	 1 coverage found on SONAR CLOUD.
===== Project spatialos/gdk-for-unity	9/9
page 1
***** Project spatialos_gdk-for-u

In [158]:
def mineOnCoverallsByBuild(project):
    commits = getBuildCommits(project)
    found = 0
    
    for commit in commits:
        build = getBuildOnCoveralls(commit[0])
        
        if build is not None:
            
            if build['covered_percent'] is None:
                print('Covered_percent Unknown: {}'.format(commit[0]))
            else:
                insertCoverage(project,build['created_at'],build['covered_percent'],commit[0])

                updateBuildCovFlag(project,commit[0])
                found += 1
                
        #else:
        #    print('Commit not found: {}'.format(commit[0]))
        
    updateProjectCovFlag(project)
    print('***** Project {}\t {}/{} builds found.'.format(project,found,len(commits)))
        

In [168]:
def mineOnSonarCloud(project,sonar_id):
    page=1
    found = 0
    if sonar_id is None:
        project = project.replace('/','_')
    else:
        project = sonar_id
        
    r = getCoverageOnSonar(project,page)
    qty_pages = 1+(r['paging']['total']/r['paging']['pageSize'])
    
    while page < qty_pages:
        print('page {}'.format(page))
        if page > 1:
            r = getCoverageOnSonar(project,page)
            
        for metric in r['measures'][0]['history']:
            if 'value' in metric.keys():
                found += 1
                insertCoverage(project,metric['date'],metric['value'],None)
                #print('SONAR CLOUD--> {} - {}'.format(metric['date'],metric['value']))
            
        page +=1
        
    updateProjectCovFlag(project)
    print('***** Project {}\t {} coverage found on SONAR CLOUD.'.format(project,found))

In [164]:
def mineOnCoveralls(project):
    page=1
    found = 0
    r = getCoverageOnCoveralls(project,page)
    qty_pages = r['pages']*2
    
    while page < qty_pages:
        #print('page {}'.format(page))
        if page > 1:
            r = getCoverageOnCoveralls(project,page)
            
        for metric in r['builds']:
            #if 'covered_percent' in metric.keys():
            found += 1
            insertCoverage(project,metric['created_at'],metric['covered_percent'],metric['commit_sha'])
            #print('mineOnCoveralls --> {} - {}'.format(metric['created_at'],metric['covered_percent']))

        page +=1
        
    updateProjectCovFlag(project)
    print('***** Project {}\t {} builds found.'.format(project,found))

In [161]:
mineOnCoveralls('i18next/i18next')
#r = getBuildOnCoveralls('9459c1749576df2e6909ac4f9dcf20ecbf641fba')
#if r is None or r['covered_percent'] is None:
#    print('oh...')
#r

#r=getCoverageOnSonar('coala_coala',1)
#r
#
#mineOnSonarCloud('coala_coala')

#r=getCoverageOnCoveralls('i18next/i18next',1)
#r

page 1
mineOnCoveralls --> 2021-11-04T09:54:06Z - 97.1467391304348
mineOnCoveralls --> 2021-11-02T06:10:26Z - 90.8707865168539
mineOnCoveralls --> 2021-11-02T05:58:30Z - 90.7988587731812
mineOnCoveralls --> 2021-11-02T05:58:15Z - 90.7988587731812
mineOnCoveralls --> 2021-11-02T05:56:42Z - 98.6428571428571
page 2
mineOnCoveralls --> 2021-11-02T05:53:12Z - 90.7988587731812
mineOnCoveralls --> 2021-11-02T05:52:46Z - 98.6457590876693
mineOnCoveralls --> 2021-11-02T03:42:25Z - 90.7988587731812
mineOnCoveralls --> 2021-11-02T03:40:10Z - 91.0967741935484
mineOnCoveralls --> 2021-11-02T03:39:42Z - 90.7659269863994
page 3
mineOnCoveralls --> 2021-11-01T20:55:26Z - 90.7681465821001
mineOnCoveralls --> 2021-11-01T20:46:24Z - 90.7746478873239
mineOnCoveralls --> 2021-11-01T20:43:44Z - 90.7746478873239
mineOnCoveralls --> 2021-10-26T02:56:39Z - 90.7725321888412
mineOnCoveralls --> 2021-10-26T02:01:03Z - 90.7659269863994
page 4
mineOnCoveralls --> 2021-10-26T02:00:07Z - 90.7659269863994
mineOnCovera

mineOnCoveralls --> 2021-10-15T21:30:17Z - 90.7659269863994
mineOnCoveralls --> 2021-10-15T21:09:03Z - 90.7659269863994
mineOnCoveralls --> 2021-10-15T17:04:51Z - 90.7659269863994
mineOnCoveralls --> 2021-10-15T17:01:11Z - 90.7659269863994
mineOnCoveralls --> 2021-10-15T17:00:22Z - 90.7659269863994
page 18
mineOnCoveralls --> 2021-10-13T20:52:30Z - 90.7659269863994
mineOnCoveralls --> 2021-10-13T20:51:57Z - 90.7659269863994
mineOnCoveralls --> 2021-10-13T20:50:54Z - 98.6524822695036
mineOnCoveralls --> 2021-10-13T20:40:41Z - 98.6281588447653
mineOnCoveralls --> 2021-10-13T20:38:47Z - 96.6878083157153
page 19
mineOnCoveralls --> 2021-10-13T20:26:42Z - 98.7341772151899
mineOnCoveralls --> 2021-10-13T20:24:51Z - 94.6685878962536
mineOnCoveralls --> 2021-10-13T20:23:51Z - 97.4751338944147
mineOnCoveralls --> 2021-10-12T12:45:50Z - 90.7659269863994
mineOnCoveralls --> 2021-10-12T12:44:54Z - 90.7659269863994
page 20
mineOnCoveralls --> 2021-10-12T12:43:43Z - 94.2238267148015
mineOnCoveralls 

mineOnCoveralls --> 2021-09-16T13:22:31Z - 90.4612978889758
mineOnCoveralls --> 2021-09-16T13:21:48Z - 90.4612978889758
mineOnCoveralls --> 2021-09-16T13:20:25Z - 90.4612978889758
mineOnCoveralls --> 2021-09-16T13:18:21Z - 90.4612978889758
mineOnCoveralls --> 2021-09-16T13:17:30Z - 90.4612978889758
page 44
mineOnCoveralls --> 2021-09-16T13:16:35Z - 90.4612978889758
mineOnCoveralls --> 2021-09-16T13:13:58Z - 90.211433046202
mineOnCoveralls --> 2021-09-16T13:06:09Z - 90.211433046202
mineOnCoveralls --> 2021-09-16T12:35:35Z - 90.4612978889758
mineOnCoveralls --> 2021-09-16T12:33:22Z - 90.4612978889758
page 45
mineOnCoveralls --> 2021-09-16T12:28:31Z - 90.4612978889758
mineOnCoveralls --> 2021-09-16T12:26:44Z - 90.4612978889758
mineOnCoveralls --> 2021-09-16T12:19:57Z - 90.4612978889758
mineOnCoveralls --> 2021-09-16T12:13:03Z - 90.4463586530932
mineOnCoveralls --> 2021-09-16T11:47:35Z - 90.4463586530932
page 46
mineOnCoveralls --> 2021-09-16T11:44:59Z - 90.4388714733542
mineOnCoveralls --

mineOnCoveralls --> 2021-08-06T08:28:58Z - 93.2156133828996
mineOnCoveralls --> 2021-08-06T07:46:25Z - 93.2156133828996
mineOnCoveralls --> 2021-08-04T17:27:38Z - 93.2407407407408
mineOnCoveralls --> 2021-08-04T12:18:48Z - 93.2156133828996
mineOnCoveralls --> 2021-08-04T09:24:46Z - 93.2156133828996
page 70
mineOnCoveralls --> 2021-08-04T09:01:20Z - 93.2282003710575
mineOnCoveralls --> 2021-08-04T08:59:37Z - 93.2282003710575
mineOnCoveralls --> 2021-08-04T07:21:57Z - 93.2093023255814
mineOnCoveralls --> 2021-08-04T07:06:17Z - 93.2344763670065
mineOnCoveralls --> 2021-08-04T07:05:25Z - 93.2344763670065
page 71
mineOnCoveralls --> 2021-08-04T07:04:30Z - 93.2282003710575
mineOnCoveralls --> 2021-08-04T07:02:40Z - 93.22191272052
mineOnCoveralls --> 2021-08-04T07:01:53Z - 93.2093023255814
mineOnCoveralls --> 2021-08-04T07:01:08Z - 93.22191272052
mineOnCoveralls --> 2021-08-04T06:58:50Z - 93.2156133828996
page 72
mineOnCoveralls --> 2021-08-04T06:57:03Z - 93.2156133828996
mineOnCoveralls --> 

mineOnCoveralls --> 2021-06-30T22:08:47Z - 93.5355147645651
mineOnCoveralls --> 2021-06-30T22:08:42Z - 93.5355147645651
mineOnCoveralls --> 2021-06-30T22:08:38Z - 93.5355147645651
mineOnCoveralls --> 2021-06-30T22:08:37Z - 93.5355147645651
mineOnCoveralls --> 2021-06-30T22:08:26Z - 93.5355147645651
page 96
mineOnCoveralls --> 2021-06-30T22:07:49Z - 93.5355147645651
mineOnCoveralls --> 2021-06-30T22:07:42Z - 93.5355147645651
mineOnCoveralls --> 2021-06-30T22:07:37Z - 93.5355147645651
mineOnCoveralls --> 2021-06-30T22:07:35Z - 93.5355147645651
mineOnCoveralls --> 2021-06-30T22:07:23Z - 93.5355147645651
page 97
mineOnCoveralls --> 2021-06-30T22:07:02Z - 93.5355147645651
mineOnCoveralls --> 2021-06-30T22:06:27Z - 93.5355147645651
Error while inserting into PostgreSQL. insertCoverage >>> Project: i18next/i18next, Date: 2021-06-30T22:06:27Z Commit: abc4a514eaf531c2b7a31c8f04c7328cb5c87fac
duplicate key value violates unique constraint "coverage_pkey"
DETAIL:  Key (repo_name, created)=(i18nex

...

mineOnCoveralls --> 2016-05-09T09:01:13Z - 66.83544303797468
mineOnCoveralls --> 2016-05-09T09:00:45Z - 66.83544303797468
mineOnCoveralls --> 2016-05-09T08:55:42Z - 66.79438058748404
mineOnCoveralls --> 2016-05-09T08:53:36Z - 66.79438058748404
mineOnCoveralls --> 2016-05-09T08:31:51Z - 66.83544303797468
page 813
mineOnCoveralls --> 2016-05-09T08:24:09Z - 66.79438058748404
mineOnCoveralls --> 2016-05-09T08:23:20Z - 66.79438058748404
mineOnCoveralls --> 2016-05-09T08:18:19Z - 66.79438058748404
mineOnCoveralls --> 2016-05-09T08:16:37Z - 66.79438058748404
mineOnCoveralls --> 2016-05-09T08:15:24Z - 66.79438058748404
page 814
mineOnCoveralls --> 2016-05-09T08:14:24Z - 66.79438058748404
mineOnCoveralls --> 2016-05-09T08:08:02Z - 66.79438058748404
mineOnCoveralls --> 2016-05-09T08:02:07Z - 66.79438058748404
mineOnCoveralls --> 2016-05-09T07:59:45Z - 66.79438058748404
mineOnCoveralls --> 2016-05-09T07:54:45Z - 67.30523627075351
page 815
mineOnCoveralls --> 2016-05-09T07:54:15Z - 66.794380587484

mineOnCoveralls --> 2016-04-03T14:39:49Z - 64.86486486486487
mineOnCoveralls --> 2016-04-01T18:09:36Z - 64.86486486486487
mineOnCoveralls --> 2016-04-01T18:06:21Z - 64.86486486486487
mineOnCoveralls --> 2016-04-01T11:44:43Z - 64.86486486486487
mineOnCoveralls --> 2016-04-01T11:44:28Z - 64.86486486486487
page 840
mineOnCoveralls --> 2016-04-01T11:43:54Z - 64.86486486486487
mineOnCoveralls --> 2016-04-01T11:39:34Z - 64.86486486486487
mineOnCoveralls --> 2016-04-01T11:38:12Z - 64.72222222222223
mineOnCoveralls --> 2016-04-01T11:37:43Z - 64.72222222222223
mineOnCoveralls --> 2016-04-01T00:00:42Z - 64.85753052917232
page 841
mineOnCoveralls --> 2016-03-31T22:14:29Z - 64.85753052917232
mineOnCoveralls --> 2016-03-29T16:55:08Z - 64.81732070365359
mineOnCoveralls --> 2016-03-29T02:31:38Z - 64.85753052917232
mineOnCoveralls --> 2016-03-29T01:59:04Z - 64.85753052917232
mineOnCoveralls --> 2016-03-29T00:14:26Z - 64.85753052917232
page 842
mineOnCoveralls --> 2016-03-28T21:52:51Z - 64.857530529172

mineOnCoveralls --> 2016-02-24T17:11:37Z - 64.79663394109397
mineOnCoveralls --> 2016-02-24T17:10:57Z - 64.79663394109397
mineOnCoveralls --> 2016-02-24T17:09:44Z - 64.79663394109397
mineOnCoveralls --> 2016-02-24T17:07:54Z - 64.79663394109397
mineOnCoveralls --> 2016-02-24T16:58:12Z - 64.79663394109397
page 867
mineOnCoveralls --> 2016-02-24T16:56:13Z - 64.79663394109397
mineOnCoveralls --> 2016-02-23T23:32:48Z - 64.79663394109397
mineOnCoveralls --> 2016-02-23T23:31:11Z - 64.79663394109397
mineOnCoveralls --> 2016-02-23T23:29:12Z - 64.79663394109397
mineOnCoveralls --> 2016-02-23T23:24:24Z - 64.79663394109397
page 868
mineOnCoveralls --> 2016-02-23T23:23:41Z - 64.79663394109397
mineOnCoveralls --> 2016-02-23T23:21:12Z - 64.79663394109397
mineOnCoveralls --> 2016-02-23T23:16:54Z - 64.79663394109397
mineOnCoveralls --> 2016-02-23T23:06:01Z - 64.79663394109397
mineOnCoveralls --> 2016-02-23T23:05:44Z - 64.79663394109397
page 869
mineOnCoveralls --> 2016-02-22T20:47:58Z - 64.796633941093

mineOnCoveralls --> 2016-01-18T16:02:22Z - 64.58923512747876
page 893
mineOnCoveralls --> 2016-01-18T16:02:05Z - 64.58923512747876
mineOnCoveralls --> 2016-01-16T02:40:55Z - 64.58923512747876
mineOnCoveralls --> 2016-01-16T02:40:38Z - 64.58923512747876
mineOnCoveralls --> 2016-01-14T14:40:38Z - 64.58923512747876
mineOnCoveralls --> 2016-01-14T14:40:15Z - 64.58923512747876
page 894
mineOnCoveralls --> 2016-01-14T14:40:09Z - 64.58923512747876
mineOnCoveralls --> 2016-01-14T11:47:29Z - 64.58923512747876
mineOnCoveralls --> 2016-01-14T11:47:14Z - 64.58923512747876
mineOnCoveralls --> 2016-01-14T11:46:43Z - 64.58923512747876
mineOnCoveralls --> 2016-01-14T10:47:50Z - 64.58923512747876
page 895
mineOnCoveralls --> 2016-01-14T08:26:48Z - 64.48863636363636
mineOnCoveralls --> 2016-01-14T08:26:37Z - 64.48863636363636
mineOnCoveralls --> 2016-01-12T21:16:50Z - 64.48863636363636
mineOnCoveralls --> 2016-01-12T21:11:52Z - 64.48863636363636
mineOnCoveralls --> 2016-01-12T19:22:30Z - 64.488636363636

mineOnCoveralls --> 2016-01-05T12:08:05Z - 72.97570850202429
mineOnCoveralls --> 2016-01-05T11:42:48Z - 72.97570850202429
mineOnCoveralls --> 2016-01-05T11:20:23Z - 72.97570850202429
***** Project i18next/i18next	 4543 builds found.


### Aux Functions

In [3]:
import psycopg2
import os
import requests 
import time
import pytz    
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd
from scipy import stats
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [4]:
def connectDB():
    f = open('/home/psql_pwd.txt', "r")
    pwd = f.readline().replace('\n','')
    
    return psycopg2.connect(user = "ci_quality",
                              password = pwd,
                              host = "127.0.0.1",
                              port = "5432",
                              database = "Causal_CI_Quality_v3")

In [54]:
def getProjects():
    query = """SELECT repo_name From PROJECTS 
    WHERE (prs_selected IS TRUE and issues_selected is true) AND CI_SERVICE ILIKE 'TRAVIS CI'"""

    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    return result

In [170]:
def getProjectsToMineCoverage():
    query = """SELECT repo_name,COV_SERVICE,sonar_id From PROJECTS 
    WHERE (prs_selected IS TRUE and issues_selected is true) 
    AND (COV_SERVICE IN ('SONAR','BOTH')  OR (LAST_BUILD IS NOT NULL AND COV_SERVICE ilike 'COVERALLS'))
    AND coverage_mined is not True
    order by repo_name;"""

    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    return result

In [61]:
def searchOnSonar(project):
    project = project.replace('/','_')
    #project = project.replace('/','-')
    son='https://sonarcloud.io/api/measures/search_history?component={}&metrics=coverage'.format(project)

    r = requestAPI(son)
    if r is not None and 'measures' in r:
        return True

    return False

In [13]:
def searchOnCoveralls(project):
    cov='https://coveralls.io/github/{}.json'.format(project)

    r = requestAPI(cov)
    
    if r is not None and 'covered_percent' in r:
        return True

    return False

In [85]:
def getBuildCommits(project):
    query = """select commit from builds_mined where repo_name like %s and coverage_mined is not True;"""

    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query, [project])
    result = cursor.fetchall()
    connection.close()
    return result

In [6]:
def updateProjectMetric(repo_name,service):
    query = """UPDATE  projects 
        set cov_service = %s
        WHERE repo_name like %s;"""

    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query, [service,repo_name])
    connection.commit()
    cursor.close()
    connection.close()

In [103]:
def updateProjectCovFlag(repo_name):
    query = """UPDATE  projects 
        set coverage_mined = True
        WHERE repo_name like %s;"""

    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query, [repo_name])
    connection.commit()
    cursor.close()
    connection.close()

In [95]:
def updateBuildCovFlag(repo_name,commit):
    query = """UPDATE  BUILDS_MINED 
        set coverage_mined = True
        WHERE repo_name like %s AND COMMIT LIKE %s;"""

    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query, [repo_name,commit])
    connection.commit()
    cursor.close()
    connection.close()

In [89]:
def insertCoverage(project,creation,coverage,commit):
    try:
        connection = connectDB()
        cursor = connection.cursor()

        query = """INSERT INTO COVERAGE (repo_name,created,coverage,commit) VALUES(%s,%s,%s,%s);"""

        cursor.execute(query, [project,creation,coverage,commit])
        connection.commit()
    except psycopg2.IntegrityError as e:
        print ("Error while inserting into PostgreSQL. insertCoverage >>> Project: {}, Date: {} Commit: {}".format(project,creation,commit)) 
        print(e)
    except Exception as e:
        print ("==============================================================")
        print ("Error while processing insertCoverage >>> Exception: {}".format(e)) 
        print('Project: {}, Date: {} Commit: {}'.format(project,creation,commit))
    finally:
        cursor.close()
        connection.close()

In [108]:
def getBuildOnCoveralls(commit):
    cov='https://coveralls.io/builds/{}.json'.format(commit)

    r = requestAPI(cov)
    return r

In [151]:
def getCoverageOnCoveralls(project,page):
    cov='https://coveralls.io/github/{}.json?page={}'.format(project,page)

    r = requestAPI(cov)
    return r

In [110]:
def getCoverageOnSonar(project,page):
    son='https://sonarcloud.io/api/measures/search_history?component={}&metrics=coverage&%22from%22=1991-01-05T00:00:00%2B0200&to=2021-01-11T23:59:59%2B0200&p={}'.format(project,page)

    r = requestAPI(son)
    return r


In [48]:
tokens =['token 80665cc2fa2fcf9f6368ac32014eda5bf5083bd3', 'token 1f3a6e7d066f937c163900db140e4506a2f3dd87', 'token 1f5163feb5b40cf70542f3a4360b957d9d6a668a']
i_token =0


def requestAPI(URL):
    # api-endpoint 
    global i_token
    
    if (len(tokens)-1) > i_token:
        i_token += 1
    else:
        i_token = 0
        
    r = requests.get(url = URL, headers={'Authorization': tokens[i_token],'Accept':'application/vnd.github.cloak-preview'})
  
    if r is None or r.status_code == 404:
        return None
    else:
        r = r.json()
    
    if r is not None and len(r) > 0:
        try:
            #print('keys: {}'.format(r.keys()))
            
            if isinstance(r, dict) and 'message' in r.keys():
                #print('{} -- {}'.format(datetime.now().strftime("%H:%M:%S"),r['message']))
                if 'API rate limit exceeded' in r['message']:
                    time.sleep(600)
                    requestAPI(URL)
                else:
                    return None

            return r
        except Exception as e:
            print('\n Erro no request get: {}'.format(e))
            print(r)
    else:
        return None

In [26]:
#r = requestAPI('https://coveralls.io/github/SonarSource/sonarqube.json')
r = requestAPI('https://sonarcloud.io/api/measures/search_history?component=simgrid_simgrid&metrics=coverage')
r

{'paging': {'pageIndex': 1, 'pageSize': 100, 'total': 135},
 'measures': [{'metric': 'coverage',
   'history': [{'date': '2019-06-06T04:46:40+0200', 'value': '80.3'},
    {'date': '2019-06-25T04:42:33+0200', 'value': '80.2'},
    {'date': '2019-07-07T04:42:38+0200', 'value': '80.7'},
    {'date': '2019-10-09T04:55:01+0200', 'value': '80.5'},
    {'date': '2019-10-10T04:53:55+0200', 'value': '80.5'},
    {'date': '2019-11-06T05:46:27+0100', 'value': '80.9'},
    {'date': '2019-11-19T05:21:16+0100', 'value': '80.7'},
    {'date': '2019-11-21T05:19:23+0100', 'value': '80.8'},
    {'date': '2019-12-10T05:24:55+0100', 'value': '81.3'},
    {'date': '2019-12-10T10:59:50+0100', 'value': '81.3'},
    {'date': '2019-12-11T09:32:43+0100', 'value': '81.5'},
    {'date': '2019-12-14T05:51:06+0100', 'value': '81.5'},
    {'date': '2019-12-20T05:57:09+0100', 'value': '81.5'},
    {'date': '2019-12-24T05:01:56+0100', 'value': '81.5'},
    {'date': '2019-12-26T05:47:30+0100', 'value': '81.5'},
    {'d

In [117]:
r['measures'][0]['history']

[{'date': '2019-10-01T23:36:52+0200'},
 {'date': '2019-10-04T03:33:53+0200'},
 {'date': '2019-10-04T03:41:24+0200', 'value': '0.0'},
 {'date': '2019-10-04T05:30:16+0200'},
 {'date': '2019-10-04T05:39:22+0200', 'value': '0.0'},
 {'date': '2019-10-09T07:25:50+0200'},
 {'date': '2019-10-09T07:34:25+0200', 'value': '0.0'},
 {'date': '2019-10-09T22:54:13+0200'},
 {'date': '2019-10-09T23:03:16+0200', 'value': '0.0'},
 {'date': '2019-10-10T12:34:06+0200'},
 {'date': '2019-10-10T12:42:09+0200', 'value': '0.0'},
 {'date': '2019-10-10T14:30:48+0200'},
 {'date': '2019-10-10T14:38:53+0200', 'value': '0.0'},
 {'date': '2019-10-15T01:13:27+0200'},
 {'date': '2019-10-15T01:22:46+0200', 'value': '0.0'},
 {'date': '2019-10-15T13:30:47+0200'},
 {'date': '2019-10-15T13:38:54+0200', 'value': '0.0'},
 {'date': '2019-10-15T23:20:44+0200'},
 {'date': '2019-10-15T23:29:58+0200', 'value': '0.0'},
 {'date': '2019-10-15T23:51:11+0200'},
 {'date': '2019-10-16T00:00:02+0200', 'value': '0.0'},
 {'date': '2019-10-16

In [18]:
def insertPeriod(project,init_period,end_period,order,metric):
    connection = connectDB()
    try:
        query = """INSERT INTO metrics_period (repo_name,init_period,end_period,order_period,period) 
        VALUES("""+'%s,'*4+'%s);'

        cursor = connection.cursor()
        cursor.execute(query,[project,init_period,end_period,order,metric])

        connection.commit()

    except Exception as e:
        print ("Error while inserting into PostgreSQL. insertPeriod >>> Repository: {}, init: {}, finish: {}".format(repo, init_period,end_period)) 
        print(e)